In [11]:
# Import libraries
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from ppca import PPCA
from sklearn.model_selection import GridSearchCV, ShuffleSplit, cross_val_score
from sklearn.svm import SVC

In [2]:
# Set variables

filepath = '../processed'
fps = 30                    # frames per second
baseline = 2                # pre-stimulus baseline (secs)
cs_dur = 4                  # CS duration (secs); note that last 2 secs coincide with shock
cr_dur = 2                  # Length of CS period to measure CR

In [3]:
def extract_features(behavior, timestamps):
    
    # Step 1: Filter for "CS" "on" events
    cs_on_events = timestamps[(timestamps['stimulus'] == 'CS') & (timestamps['event'] == 'on')]

    # Step 2: Identify frames where stimulus is "US" and event is "on"
    us_on_frames = timestamps[(timestamps['stimulus'] == 'US') & (timestamps['event'] == 'on')]['frame'].values
    
    # Function to get sequences of frames
    def get_sequences(start_frames, offset):
        sequences = []
        for frame in start_frames:
            if offset == 1:
                sequence_frames = range(frame, frame + 60)  # Frames after the event
            elif offset == -1:
                sequence_frames = range(frame - 60, frame)  # Frames before the event

            # Exclude frames where stimulus is "US" "on"
            valid_frames = [f for f in sequence_frames if f not in us_on_frames]

            # Get dx and dy values for valid frames
            valid_behavior = behavior[behavior['frame'].isin(valid_frames)]

            # Check if we have enough frames (should be 60)
            if len(valid_behavior) >= 60:
                # Take the first 60 frames (in case we excluded some)
                valid_behavior = valid_behavior.head(60) if offset == 1 else valid_behavior.tail(60)
                # Concatenate dx and dy values
                sequence = np.hstack((valid_behavior['dx'].values, valid_behavior['dy'].values))
                sequences.append(sequence)
        
        return sequences

    # Step 3: Collect the sequences after and before each "CS" "on" event
    sequences_after_cs = get_sequences(cs_on_events['frame'], 1)
    sequences_before_cs = get_sequences(cs_on_events['frame'], -1)

    # Step 4: Create numpy arrays and stack them
    result_array_after = np.array(sequences_after_cs)
    result_array_before = np.array(sequences_before_cs)
    features = np.vstack((result_array_after, result_array_before))

    # Step 5: Create labels array
    labels_after = np.ones(result_array_after.shape[0])
    labels_before = np.zeros(result_array_before.shape[0])
    labels = np.hstack((labels_after, labels_before))
    
    return features, labels

In [4]:
# Load data

def load_data(filepath):
    
    Features = np.empty((0,120))
    Labels = np.empty((0,1))
    for f in os.listdir(filepath):
        if f.endswith("_timestamps.csv"):
            # Load timestamps
            timestamps = pd.read_csv(os.path.join(filepath,f))
            timestamps['event'] = timestamps['event'].apply(lambda x: x.lower() if isinstance(x, str) else x) # get rid of capitalization
            
            # Load behavior
            p = f.split("_")
            g = "_".join([p[0],p[1],'behavior.csv'])
            behavior = pd.read_csv(os.path.join(filepath,g))
            behavior['dx'] = behavior.groupby('ID')['x'].diff()
            behavior['dy'] = behavior.groupby('ID')['y'].diff()
            behavior.fillna(0)
            
            features, labels = extract_features(behavior,timestamps)
            
            Features = np.append(Features,features,axis=0)
            Labels = np.append(Labels,labels)
            
    return Features, Labels

In [5]:
# Load data and extract features
Features, Labels = load_data(filepath)

In [13]:
# Reduce dimensionality using probabilistic PCA
Features = StandardScaler().fit_transform(Features)     # Z-score first
ppca = PPCA()
ppca.fit(data=Features, d=20, verbose=False)
Components = ppca.transform()

In [14]:
# Classify trials as US vs. ITI
clf = SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, Components, Labels, cv=5)
scores

array([0.48837209, 0.50581395, 0.53488372, 0.47674419, 0.48255814])

In [16]:
C_range = np.logspace(-2, 10, 4)
gamma_range = np.logspace(-9, 3, 4)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(kernel='rbf'), param_grid=param_grid, cv=cv)
grid.fit(Components, Labels)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

The best parameters are {'C': 1000000.0, 'gamma': 0.1} with a score of 0.55
